In [1]:
import os
import numpy as np
import pandas as pd

from infodynamics.metrics import cos_sim, js_dist
from src.dataset.util import read_jsonl, write_jsonl


DATA_PROCESSED = '../data/processed/'
DATA_INTERIM = '../data/interim/'

/Users/au582299/Repositories/friths/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
publications = read_jsonl(os.path.join(DATA_PROCESSED, 'publication.jsonl'))
df_pub = pd.DataFrame.from_dict(publications)
df_pub['doc_id'] = df_pub.apply(lambda x: x['id'].split('_')[0])

In [ ]:
distances = []
for i in publications['doc_id']:
    abs = publications.filter(lambda x: x['id'] == str(i) + "_a")
    bod = publications.filter(lambda x: x['id'] == str(i) + "_b")
    dist = cos_sim(
        np.array(abs['embedding']), 
        np.array(bod['embedding'])
        )
    distances.append(dist)

In [ ]:
meta = pd.read_csv(os.path.join(DATA_INTERIM, 'publications_meta.csv'))
meta['distance'] = distances
meta['distance'].plot()